# Data --> CSP-Structure

## How should the Data-Structure look like? 

It's a Pandas Dataframe with all relevant Information. 
| id | size | variables | domains | constraints | 
|----|------|-----------|---------|-------------|
| lgp-test-5x6-16 | 5*6 | ['name', 'nationalities', 'book genres', 'lunch', 'color', 'animals'] | {'name': [`Peter`, `Alice`, `Bob`, `Eric`, `Arnold`], 'nationalities': [`norwegian`, `german`, `dane`, `brit`, `swede`], 'book genres': [`fantasy`, `biography`, `romance`, `mystery`, `science fiction`], 'lunch': [`stir fry`, `grilled cheese`, `pizza`, `spaghetti`, `stew`], 'color': [`red`, `green`, `blue`, `yellow`, `white`], 'animals': [`bird`, `dog`, `cat`, `horse`, `fish`]} | [IsContraint('fantasy', 'norwegian'), NextToConstraint('cat', 'biography'), IsConstraint('German', 'Bob'), IsConstraint('yellow', 'Bob'), IsConstraint('Green', 'Peter'), LeftOrRightConstraint('Dane', 'Pizza', distance=1), LeftConstraint('blue', 'Dane'), LeftConstraint('grilled Cheese', 'Norwegian'), ..., LeftConstraint('fish', 'science fiction', distance=0), ...,  RightConstraint(x,y), RightConstraint(x,y, distance=0)] |

In [1]:
import pandas as pd
import numpy as np

df = pd.read_parquet("data/Gridmode-00000-of-00001.parquet")
for i in df.puzzle[0].splitlines():
    print(i)
print(df.solution[0])

There are 5 houses, numbered 1 to 5 from left to right, as seen from across the street. Each house is occupied by a different person. Each house has a unique attribute for each of the following characteristics:
 - Each person has a unique name: `Peter`, `Alice`, `Bob`, `Eric`, `Arnold`
 - The people are of nationalities: `norwegian`, `german`, `dane`, `brit`, `swede`
 - People have unique favorite book genres: `fantasy`, `biography`, `romance`, `mystery`, `science fiction`
 - Everyone has something unique for lunch: `stir fry`, `grilled cheese`, `pizza`, `spaghetti`, `stew`
 - Each person has a favorite color: `red`, `green`, `blue`, `yellow`, `white`
 - The people keep unique animals: `bird`, `dog`, `cat`, `horse`, `fish`

## Clues:
1. The person who loves fantasy books is the Norwegian.
2. The cat lover and the person who loves biography books are next to each other.
3. The German is Bob.
4. The person who loves yellow is Bob.
5. The person whose favorite color is green is Peter.
6. 